# Example: Spark + DSE

In BGDS, we advocate for the convination of the following tools as the best solution to address Big data's problems.

- Apache Spark
- Apache Cassandra (DSE)
- Apache Solr (DSE)

This notebook is an example of how to use these tools together to analyze the data crawled by the `Bovespa` example crawler included in the DaVinci distribution ().

In [31]:
import IPython
IPython.auto_scroll_threshold = 9999

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
from IPython.display import display, HTML

%load_ext autotime

The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
time: 1.1 ms


In [32]:
# Configurations related to Cassandra connector & Cluster
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = \
    '--packages datastax:spark-cassandra-connector:2.4.0-s_2.11 pyspark-shell'

time: 381 µs


In [33]:
# Creating Spark Context
from pyspark import SparkContext, SparkConf

conf = SparkConf() \
 .setAppName("Factor Analysis Job") \
 .set("spark.cassandra.connection.host", "tfm_uoc_dse") \
 .set("spark.sql.dse.search.enableOptimization", "on") 
# .set("spark.cassandra.auth.username", "cassandra") \
# .set("spark.cassandra.auth.password", "*********")  

spark_master = "local"
sc = SparkContext(spark_master, "Factor Analysis Job", conf=conf)

ValueError: Cannot run multiple SparkContexts at once; existing SparkContext(app=Factor Analysis Job, master=local) created by __init__ at <ipython-input-3-1410226d81b3>:13 

time: 51.5 ms


In [34]:
# Creating PySpark SQL Context
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

time: 1.43 ms


In [35]:
# Utility function for return a data frame attached to the informed keyspace.table
def load_and_get_table_df(keys_space_name, table_name):
    table_df = sqlContext.read \
        .format("org.apache.spark.sql.cassandra") \
        .options(table=table_name, keyspace=keys_space_name) \
        .option("spark.sql.dse.search.enableOptimization", "on") \
        .load()
    return table_df

time: 504 µs


In [43]:
from pyspark.sql.functions import udf, col

# Loading Bovespa companies table
companies = load_and_get_table_df("tfm_uoc", "bovespa_company")
companies.filter(~col("situation").eqNullSafe("CANCELED")).show(10)

+-----------+-----+-------------+------------------+--------------------+------------+--------------------+--------------+------------+----------+---------+----------+--------------------+
|entity_type| ccvm|canceled_date|              cnpj|        company_name|company_type|          created_at|deleted_reason|granted_date|is_deleted|situation|solr_query|          updated_at|
+-----------+-----+-------------+------------------+--------------------+------------+--------------------+--------------+------------+----------+---------+----------+--------------------+
|    company| 1023|         null|00.000.000/0001-91|BANCO DO BRASIL S.A.|CIAS ABERTAS|2019-12-26 02:11:...|          null|  1977-07-20|     false|  GRANTED|      null|2019-12-26 02:11:...|
|    company|10243|         null|83.296.889/0001-23|MASSA FALIDA DA S...|CIAS ABERTAS|2019-12-26 02:12:...|          null|  1979-11-30|     false|  GRANTED|      null|2019-12-26 02:12:...|
|    company|10456|         null|61.079.117/0001-05|   

In [42]:
# Check the execution plan of the query
companies.filter(~col("situation").eqNullSafe("CANCELED")).explain(True)

== Parsed Logical Plan ==
'Filter NOT ('situation <=> CANCELED)
+- Relation[entity_type#1011,ccvm#1012,canceled_date#1013,cnpj#1014,company_name#1015,company_type#1016,created_at#1017,deleted_reason#1018,granted_date#1019,is_deleted#1020,situation#1021,solr_query#1022,updated_at#1023] org.apache.spark.sql.cassandra.CassandraSourceRelation@3e768c0f

== Analyzed Logical Plan ==
entity_type: string, ccvm: string, canceled_date: date, cnpj: string, company_name: string, company_type: string, created_at: timestamp, deleted_reason: string, granted_date: date, is_deleted: boolean, situation: string, solr_query: string, updated_at: timestamp
Filter NOT (situation#1021 <=> CANCELED)
+- Relation[entity_type#1011,ccvm#1012,canceled_date#1013,cnpj#1014,company_name#1015,company_type#1016,created_at#1017,deleted_reason#1018,granted_date#1019,is_deleted#1020,situation#1021,solr_query#1022,updated_at#1023] org.apache.spark.sql.cassandra.CassandraSourceRelation@3e768c0f

== Optimized Logical Plan ==
F

In [10]:
from pyspark.sql.functions import *
from pyspark.sql import Window

# Accessing to the fundamental data of the companies
# Attaching a data frame to the accounting notes available in the database
fundamentals = load_and_get_table_df("tfm_uoc", "bovespa_account")

# Get the Capital Expenditures from the Fundamentals
# Account 1    = Total Assets of the company
# Account 6.02 = Capital Expenditure reported by the company during the period
factors_df = fundamentals.filter(
    col("number").isin(["6.02", "1"]))
factors_df = factors_df.withColumn(
    "factor_name", when(factors_df.number == "1", "total_assets").otherwise("cap_ex_reported"))

factors_df = factors_df\
    .select(col("ccvm").alias("asset"), 
            col("period").alias("astodate"),
            col("factor_name"), 
            col("amount").alias("amount"))

factors_df = factors_df.groupby(col("asset"), col("astodate"))\
    .pivot("factor_name").sum("amount").orderBy("asset", "astodate")

factors_df = factors_df.withColumn(
    "cap_ex_reported_scaled", 
    when(factors_df.cap_ex_reported > 0, 0).otherwise(abs(factors_df.cap_ex_reported)) / factors_df.total_assets)

factors_df = factors_df.withColumn("capex_vol_6q", stddev(col("cap_ex_reported_scaled"))
             .over(Window.partitionBy("asset").rowsBetween(-5, 0)) )

factors_df = factors_df.withColumn("capex_vol_6q_ranked", rank()\
             .over(Window.partitionBy("astodate").orderBy(asc("capex_vol_6q"))))

AttributeError: 'DataFrame' object has no attribute 'cap_ex_reported'

In [ ]:
factors_df.select(col("astodate"), col("asset"), col("capex_vol_6q"), col("capex_vol_6q_ranked")).show(100)

In [ ]:
factors_df.cache()

In [ ]:
def z_score_w(col, w):
    avg_ = avg(col).over(w)
    avg_sq = avg(col * col).over(w)
    sd_ = sqrt(avg_sq - avg_ * avg_)
    return (col - avg_) / sd_

w = Window().partitionBy("astodate")
factors_df_standard = factors_df.withColumn("capex_vol_6q_ranked_zscored", 
                                            z_score_w(factors_df.capex_vol_6q_ranked, w))

In [ ]:
factors_df_standard.select(
    col("astodate"), 
    col("asset"), 
    col("capex_vol_6q_ranked_zscored")).show(100)

In [ ]:
factors_data = factors_df_standard.select(
    col("astodate"), 
    col("asset"), 
    col("capex_vol_6q_ranked_zscored").alias("capex_vol_6q")).limit(10).toPandas()

# Persist the results

Now it's time to save the factor into a table. To do this, the system should had create a 
keyspace for us in the database. The keyspace should have the same TOKEN than the one used
to connect through the API.

Each user will have it's own space in the system to run the analyses.

```sql
CREATE KEYSPACE IF NOT EXISTS <USER_ID>_analysis WITH replication = {'class': 'SimpleStrategy', 'replication_factor' : 1};
```

with full access to it:

```sql
-- Create the user using the token and
CREATE ROLE user_<USER_ID> WITH PASSWORD = '<USER_TOKEN>' AND LOGIN = true AND SUPERUSER = false;

-- Grant all permission for the user keyspace
GRANT ALL PERMISSIONS IN KEYSPACE <USER_ID>_analysis to user_<USER_ID>;
```

PRO features:
- Allow replication of data (ReplicationFactor = 3)
    ```sql
    CREATE KEYSPACE IF NOT EXISTS <TOKEN>_analysis WITH replication = {'class': 'SimpleStrategy', 'replication_factor' : 3};
    ```
- Network replication (multiple DataCenters)
    ```sql
    CREATE KEYSPACE IF NOT EXISTS <TOKEN>_analysis WITH replication = {'class': 'NetworkTopologyStrategy', 'DC1' : 2, 'DC2': 3, 'DC3': 2};
    ```

In [ ]:
try:
    from dse.cluster import Cluster
    from dse.auth import DSEPlainTextAuthProvider
except ImportError:
    from cassandra.cluster import Cluster
    from cassandra.auth import DSEPlainTextAuthProvider

cluster = Cluster(['tfm_uoc_dse'])  # provide contact points and port
session = cluster.connect('xalperte_analysis')

In [ ]:
try:
    from dse.cqlengine.models import Model
    from dse.cqlengine import columns
    from dse.cqlengine.management import sync_table
    from dse.cqlengine.connection import register_connection
except ImportError:
    from cassandra.cqlengine.models import Model
    from cassandra.cqlengine import columns
    from cassandra.cqlengine.management import sync_table
    from cassandra.cqlengine.connection import register_connection
    
from pyspark.sql.types import StringType, DateType, TimestampType
from pyspark.sql.types import FloatType, DecimalType, DoubleType
from pyspark.sql.types import IntegerType

def create_cassandra_model(schema, table_name, primary_colums=None):
    class MyModel(object):
        pass

    if not isinstance(primary_colums, list):
        primary_colums = [primary_colums]
        
    df_fields = {}
    for field in schema.fields:
        df_fields[field.name] = field
        
    metadata = {
        "__table_name__": table_name
    }
        
    # First the primary keys
    for field_name in primary_colums:
        field = df_fields[field_name]
        if isinstance(field.dataType, StringType):
            metadata[field_name] = columns.Text(required=(not field.nullable), primary_key=True)
        if isinstance(field.dataType, DateType):
            metadata[field_name] = columns.Date(required=(not field.nullable), primary_key=True)
        if isinstance(field.dataType, TimestampType):
            metadata[field_name] = columns.DateTime(required=(not field.nullable), primary_key=True)
        if isinstance(field.dataType, DoubleType):
            metadata[field_name] = columns.Double(required=(not field.nullable), primary_key=True)
        if isinstance(field.dataType, DecimalType):
            metadata[field_name] = columns.Decimal(required=(not field.nullable), primary_key=True)
        if isinstance(field.dataType, FloatType):
            metadata[field_name] = columns.Float(required=(not field.nullable), primary_key=True)
        if isinstance(field.dataType, IntegerType):
            metadata[field_name] = columns.Integer(required=(not field.nullable), primary_key=True)

    # First the primary keys
    for field_name, field in df_fields.items():
        if field_name not in primary_colums:
            if isinstance(field.dataType, StringType):
                metadata[field_name] = columns.Text(required=(not field.nullable), primary_key=False)
            if isinstance(field.dataType, DateType):
                metadata[field_name] = columns.Date(required=(not field.nullable), primary_key=False)
            if isinstance(field.dataType, TimestampType):
                metadata[field_name] = columns.DateTime(required=(not field.nullable), primary_key=False)
            if isinstance(field.dataType, DoubleType):
                metadata[field_name] = columns.Double(required=(not field.nullable), primary_key=False)
            if isinstance(field.dataType, DecimalType):
                metadata[field_name] = columns.Decimal(required=(not field.nullable), primary_key=False)
            if isinstance(field.dataType, FloatType):
                metadata[field_name] = columns.Float(required=(not field.nullable), primary_key=False)
            if isinstance(field.dataType, IntegerType):
                metadata[field_name] = columns.Integer(required=(not field.nullable), primary_key=False)

    return type('MyModel', (Model,), metadata)
            
            
model = create_cassandra_model(factors_df_standard.schema, "capex_vol_6q", "asset")
register_connection("my_connection", session=session)
sync_table(model, keyspaces=["xalperte_analysis"], connections=["my_connection"])

In [ ]:
factors_df.write\
    .format("org.apache.spark.sql.cassandra")\
    .options(table="capex_vol_6q", keyspace="xalperte_analysis")\
    .option("confirm.truncate","true")\
    .mode("overwrite")\
    .partitionBy("astodate")\
    .save()

In [ ]:
factors_data.set_index(['astodate', 'asset'], inplace=True)
display(factors_data)

In [ ]:
import pandas as pd

unixt_factors_data = factors_data.set_index(pd.MultiIndex.from_tuples(
    [(x.timestamp(), y) for x, y in factors_data.index.values],
    names=['astodate', 'asset']))
display(unixt_factors_data)

In [ ]:
sc.stop()